<a href="https://colab.research.google.com/github/iAmanSharan/Face-mask-detection/blob/main/Face_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPooling2D , Flatten

In [ ]:
pip install kaggle

In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()



In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

In [4]:
import zipfile

# Replace 'dataset-name' with the actual name of the dataset
with zipfile.ZipFile('face-mask-12k-images-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('path/to/extract')

# Now the dataset files are in the 'path/to/extract' directory


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(rescale=1./255)

# Define the directories for the training, validation, and test data
train_dir = '/content/path/to/extract/Face Mask Dataset/Train'
validation_dir = '/content/path/to/extract/Face Mask Dataset/Validation'
test_dir = '/content/path/to/extract/Face Mask Dataset/Test'

# Create generators for the training, validation, and test data
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


In [13]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add the output layer
predictions = Dense(1, activation='sigmoid' )(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Compile your model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Train your model
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)
